In [1]:
import os
import sys

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.nn.init

from torchsummary import summary

from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(123)

cuda


In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = datasets.MNIST(root      = 'data/',
                             train     = True,
                             transform = transforms.ToTensor(),
                             download  = True)

mnist_test  = datasets.MNIST(root      = 'data/',
                             train     = False,
                             transform = transforms.ToTensor(),
                             download  = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [5]:
data_loader = torch.utils.data.DataLoader(dataset    = mnist_train,
                                          batch_size = batch_size,
                                          shuffle    = True,
                                          drop_last  = True)

In [6]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = CNN().to(device)

In [8]:
summary(model, input_size = (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Linear-7                   [-1, 10]          31,370
Total params: 50,186
Trainable params: 50,186
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.65
Params size (MB): 0.19
Estimated Total Size (MB): 0.84
----------------------------------------------------------------


In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print(f'[Epoch: {epoch + 1:>4}] cost = {avg_cost:>.9}')

[Epoch:    1] cost = 0.22169365
[Epoch:    2] cost = 0.0609542094
[Epoch:    3] cost = 0.0451117344
[Epoch:    4] cost = 0.0364775062
[Epoch:    5] cost = 0.0297244694
[Epoch:    6] cost = 0.0253814068
[Epoch:    7] cost = 0.0198781155
[Epoch:    8] cost = 0.0176544599
[Epoch:    9] cost = 0.0151308328
[Epoch:   10] cost = 0.0131556559
[Epoch:   11] cost = 0.0102029415
[Epoch:   12] cost = 0.00863219425
[Epoch:   13] cost = 0.00800781418
[Epoch:   14] cost = 0.00785358064
[Epoch:   15] cost = 0.0047071469


In [12]:
# for test
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)
    
    prediction = model(X_test)
    print(f"Prediction Size : {prediction.size()}")
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print(f"Accuracy : {accuracy.item()}")

Prediction Size : torch.Size([10000, 10])
Accuracy : 0.9870999455451965
